# Mapping by sequencing pipeline
## Parental strain analysis - Hawaiian tutorial
* [Amhed Missael Vargas Velazquez](https://www.researchgate.net/profile/Amhed-Vargas-Velazquez)
* Post-doctoral fellow, [SGB lab](https://syngenbio.kaust.edu.sa/), [KAUST](https://www.kaust.edu.sa/en)

## Description
This jupyter notebook will guide you through the computation used to identify genomic variants in the highly polymorphic *C. elegans* strain CB4856 which was isolated in Hawaii (hence its alias as *Hawaiian*). This step is essential in the [mapping by sequencing](https://www.genetics.org/content/204/2/451) pipeline only if the mapping strain, *a.k.a*. the parental strain, had not been analized before, *i.e.*, you don't have a .vcf file containing background genomic variants. 

## Software requirements
The core instance running this script is Python2. However, the analysis are performed by multiple programs (handled by `system calls`) which have to be installed or, for portability convenience, be present in a folder (check WGS-Software_configuration notebook for further details). Note that software in **bold** must be installed and compiled in the executing computer because they cannot be used as stand-alone versions.

* [**samtools**](http://www.htslib.org/)
* [**bwa**](https://github.com/lh3/bwa)
* [GATK](https://gatk.broadinstitute.org/hc/en-us) [3.7+](https://console.cloud.google.com/storage/browser/gatk-software/package-archive/gatk/); note that current version of GATK (4.0+) uses different nomenclature and have different tools which makes it not compatible with this pipeline.
* [picard](https://broadinstitute.github.io/picard/)
* [snpEff](https://pcingola.github.io/SnpEff/)
* [fastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/)
* [SRAToolkit](https://github.com/ncbi/sra-tools)
* wget
* zcat
* [**R**](https://cran.r-project.org/)
* Python2
* Perl

### R libraries
* ggplot2

### Prerequisites
Before running any code, make sure to select which folder where the analysis will be performed:

In [1]:
## Load libraries
#os to move within directories
import os
##Set main working directory
#Same location as script
#path = '/home/jupyter-user/'
path = os.getcwd()
##Move to path
os.chdir(path)
#Show current directory
print('Main directory:\n' + os.getcwd())

#### A reference genome
In order to run this pipeline, a *C. elegans* reference genome (WS235/ce11) is needed; you can download the most recent ensemble version (ce11) by running the following code:

In [3]:
##Create directory to place data
!mkdir -p {path}/data
##Move to that directory
os.chdir(path+'/data')
#Download the C. elegans genome from ensemble release 99 (same version as SnpEff)
!wget -q ftp://ftp.ensembl.org/pub/release-99/fasta/caenorhabditis_elegans/dna/Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz
#Uncompress and rename
!zcat Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz > Caenorhabditis_elegans.WBcel235.99.softmasked.fa


mkdir: cannot create directory ‘data’: File exists


Then it need to be indexed by samtools:

In [4]:
!samtools faidx Caenorhabditis_elegans.WBcel235.99.softmasked.fa

And bwa:

In [5]:
!bwa index Caenorhabditis_elegans.WBcel235.99.softmasked.fa

[bwa_index] Pack FASTA... 0.59 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=200572802, availableWord=26113028
[BWTIncConstructFromPacked] 10 iterations done. 43074434 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 79575970 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 112014578 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 140842098 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 166460034 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 189225266 characters processed.
[bwt_gen] Finished constructing BWT in 66 iterations.
[bwa_index] 42.12 seconds elapse.
[bwa_index] Update BWT... 0.45 sec
[bwa_index] Pack forward-only FASTA... 0.34 sec
[bwa_index] Construct SA from BWT and Occ... 21.76 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index Caenorhabditis_elegans.WBcel235.99.softmasked.fa
[main] Real time: 67.565 sec; CPU: 65.264 sec


#### Stand alone software
Most of the programs used within this pipeline have "stand-alone" versions that allow users to run their analysis on any computer they want. However, first you have to make sure to have those programs. Particularly, make sure to have a directory containing the following ones: 

- GATK (GenomeAnalysisTK.jar)
- Picard (picard.jar)
- SnpEff (folder with both snpEff.jar and SnpSift.jar, and another folder with its database; more below)

For your convenience, we distribute a folder already containing these programs. Just make sure to set properly the path to them, e.g.:

In [6]:
#Path to software folder
softpath = '/home/WGS_pipeline/Software/'

In the case that a new version of a program is needed or the software configuration has changed, please check the jupyter notebook `WGS-Software_configuration`. This notebook indicates where and how to intall all the programs needed for the pipeline.

## Introduction
Whole Genome Sequencing (WGS) has revolutionized how we identify genes that act in the development of a phenotype. For a short historical review check my [lab presentation](https://docs.google.com/presentation/d/1pAx1WlkFoawavdneXdca28Gdr7n6Yfc4EdoAf7yVIqc/edit?usp=sharing) and for a better understanding read the Wormbook chapter on [NGS methods to map mutations in *C. elegans*](https://www.genetics.org/content/204/2/451).

This jupyter notebook will guide you through the computation used to identify genomic variants in the highly polymorphic C. elegans strain CB4856 which was isolated in Hawaii (hence its alias as Hawaiian). This is the first notebook of a series of tutorials

### Hawaiian strain

#### Reads

Lets download fastq reads from SRA cloud hosted in Amazon Web Services. We would use `wget` to download the SRA file and the SRAToolkit to dump them into fastq files:

In [7]:
## SRR2003569: https://trace.ncbi.nlm.nih.gov/Traces/sra/?run=SRR2003569
!wget -q https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR2003569/SRR2003569

##Dump files into fastq files
!fastq-dump --split-files SRR2003569

##Verify if files were properly downloaded
!ls

Read 19794919 spots for SRR2003569
Written 19794919 spots for SRR2003569
Caenorhabditis_elegans.WBcel235.99.softmasked.fa
Caenorhabditis_elegans.WBcel235.99.softmasked.fa.amb
Caenorhabditis_elegans.WBcel235.99.softmasked.fa.ann
Caenorhabditis_elegans.WBcel235.99.softmasked.fa.bwt
Caenorhabditis_elegans.WBcel235.99.softmasked.fa.fai
Caenorhabditis_elegans.WBcel235.99.softmasked.fa.pac
Caenorhabditis_elegans.WBcel235.99.softmasked.fa.sa
Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz
Caenorhabditis_elegans.WBcel235.dna_sm.toplevel.fa.gz.1
SRR2003569
SRR2003569_1.fastq
SRR2003569_2.fastq


Now lets go out

In [13]:
##Move to that directory
os.chdir(path)
##Create directory to place data
!mkdir -p {path}/analysis
##Move to that directory
os.chdir( path + '/analysis' )

Lets now define parameters for pipeline:

In [21]:
##Inputs
SampleName = 'CB4856'
FastqF1 = (path + '/data/SRR2003569_1.fastq') 
FastqF2 = (path + '/data/SRR2003569_2.fastq')
ReferenceGenome = (path + '/data/Caenorhabditis_elegans.WBcel235.99.softmasked.fa')
SnpEffGen = 'WBcel235.99'

#Minimal quality for bam
minMaqQforBam = 1
#Minimum quality for vcf filtering
minMaqQforVcf = 10
#Minimum basequality for variant calling
minBaseQforVcf = 10
#
minVarCall = 10
diffStep = 10
minDel = 2
maxSample= 2

#Number of threats
Ncpu=20
#Ram for java
ramG=40

#### FastQC interpretation

In [5]:
##Create directory to place data
!mkdir -p FastQC
##Move to that directory
os.chdir(path + '/analysis/FastQC')

In [6]:
##Perform fastQC analysis
!fastqc -t {Ncpu} {FastqF1} {FastqF2} -o .

Started analysis of SRR2003569_1.fastq
Started analysis of SRR2003569_2.fastq
Approx 5% complete for SRR2003569_2.fastq
Approx 5% complete for SRR2003569_1.fastq
Approx 10% complete for SRR2003569_2.fastq
Approx 10% complete for SRR2003569_1.fastq
Approx 15% complete for SRR2003569_2.fastq
Approx 15% complete for SRR2003569_1.fastq
Approx 20% complete for SRR2003569_2.fastq
Approx 20% complete for SRR2003569_1.fastq
Approx 25% complete for SRR2003569_2.fastq
Approx 25% complete for SRR2003569_1.fastq
Approx 30% complete for SRR2003569_2.fastq
Approx 35% complete for SRR2003569_2.fastq
Approx 30% complete for SRR2003569_1.fastq
Approx 40% complete for SRR2003569_2.fastq
Approx 35% complete for SRR2003569_1.fastq
Approx 45% complete for SRR2003569_2.fastq
Approx 40% complete for SRR2003569_1.fastq
Approx 50% complete for SRR2003569_2.fastq
Approx 45% complete for SRR2003569_1.fastq
Approx 55% complete for SRR2003569_2.fastq
Approx 50% complete for SRR2003569_1.fastq
Approx 60% complete f

Basic assesment

In [7]:
##Unzip folders
!unzip -qq \*.zip
print('SRR2003569_1')
!cat SRR2003569_1_fastqc/summary.txt
print('SRR2003569_2')
!cat SRR2003569_2_fastqc/summary.txt


2 archives were successfully processed.
SRR2003569_1
PASS	Basic Statistics	SRR2003569_1.fastq
PASS	Per base sequence quality	SRR2003569_1.fastq
WARN	Per tile sequence quality	SRR2003569_1.fastq
PASS	Per sequence quality scores	SRR2003569_1.fastq
FAIL	Per base sequence content	SRR2003569_1.fastq
PASS	Per sequence GC content	SRR2003569_1.fastq
PASS	Per base N content	SRR2003569_1.fastq
PASS	Sequence Length Distribution	SRR2003569_1.fastq
WARN	Sequence Duplication Levels	SRR2003569_1.fastq
PASS	Overrepresented sequences	SRR2003569_1.fastq
WARN	Adapter Content	SRR2003569_1.fastq
SRR2003569_2
PASS	Basic Statistics	SRR2003569_2.fastq
PASS	Per base sequence quality	SRR2003569_2.fastq
PASS	Per tile sequence quality	SRR2003569_2.fastq
PASS	Per sequence quality scores	SRR2003569_2.fastq
WARN	Per base sequence content	SRR2003569_2.fastq
PASS	Per sequence GC content	SRR2003569_2.fastq
PASS	Per base N content	SRR2003569_2.fastq
PASS	Sequence Length Distribution	SRR2003569_2.fastq
WARN	Sequence Dup

As seen in the summary, our reads passed most of the metrics for quality control. You can read the full report via the following links:

<a href="analysis/FastQC/SRR2003569_1_fastqc.html">SRR2003569_1</a>

<a href="analysis/FastQC/SRR2003569_2_fastqc.html">SRR2003569_2</a>

#### Mapping

Make directory for alignements

In [21]:
##Move to analysis
os.chdir(path + '/analysis/')
##Create directory to place data
!mkdir -p Alignments
##Move to that directory
os.chdir(path + '/analysis/Alignments')

In [22]:
!bwa mem -t {Ncpu} -M {ReferenceGenome} {FastqF1} {FastqF2} > {SampleName}.sam 

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 2222224 sequences (200000160 bp)...
[M::process] read 2222224 sequences (200000160 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (171, 889710, 298, 149)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (310, 856, 4026)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 11458)
[M::mem_pestat] mean and std.dev: (2349.29, 2740.90)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 15174)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (136, 230, 283)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 577)
[M::mem_pestat] mean and std.dev: (215.07, 93.36)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 724)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 

[M::process] read 2222224 sequences (200000160 bp)...
[M::mem_process_seqs] Processed 2222224 reads in 300.068 CPU sec, 15.172 real sec
[M::process] read 2222224 sequences (200000160 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (160, 890609, 329, 163)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (277, 747, 3294)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 9328)
[M::mem_pestat] mean and std.dev: (1709.83, 2007.20)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 12345)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (126, 220, 276)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 576)
[M::mem_pestat] mean and std.dev: (207.07, 92.76)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 726)
[M::mem_pestat] analyzing insert size distribution for orien

[M::process] read 2222224 sequences (200000160 bp)...
[M::mem_process_seqs] Processed 2222224 reads in 290.592 CPU sec, 14.673 real sec
[M::process] read 2222224 sequences (200000160 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (165, 890071, 271, 180)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (276, 747, 3245)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 9183)
[M::mem_pestat] mean and std.dev: (1737.05, 2049.74)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 12152)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (131, 225, 280)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 578)
[M::mem_pestat] mean and std.dev: (211.26, 93.00)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 727)
[M::mem_pestat] analyzing insert size distribution for orien

[M::mem_process_seqs] Processed 2222224 reads in 308.648 CPU sec, 15.747 real sec
[M::process] read 2222224 sequences (200000160 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (167, 889988, 332, 174)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (331, 841, 2575)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 7063)
[M::mem_pestat] mean and std.dev: (1476.18, 1589.56)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 9307)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (135, 229, 282)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 576)
[M::mem_pestat] mean and std.dev: (214.48, 93.45)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 723)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 75) percentile: (

Now lets transform to bam

In [23]:
!samtools view -@ {Ncpu} -S -bh {SampleName}.sam | samtools sort -@ {Ncpu} - > {SampleName}.bam

[bam_sort_core] merging from 0 files and 20 in-memory blocks...


Now lets index

In [24]:
!samtools index {SampleName}.bam

Filter for mapping quality

In [25]:
!samtools view -@ {Ncpu} -q {minMaqQforBam} -bh {SampleName}.bam > {SampleName}.SF.bam

Then add and replace groups via picard

In [27]:
!java -Xmx{ramG}g -jar {softpath}/picard2.23.6/picard.jar AddOrReplaceReadGroups I={SampleName}.SF.bam O={SampleName}.RG.bam RGID={SampleName} RGLB=LB RGPL=illumina RGPU=PU RGSM={SampleName}

INFO	2021-06-17 10:54:28	AddOrReplaceReadGroups	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    AddOrReplaceReadGroups -I CB4856.SF.bam -O CB4856.RG.bam -RGID CB4856 -RGLB LB -RGPL illumina -RGPU PU -RGSM CB4856
**********


10:54:29.062 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/Software/picard2.23.6/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Thu Jun 17 10:54:29 AST 2021] AddOrReplaceReadGroups INPUT=CB4856.SF.bam OUTPUT=CB4856.RG.bam RGID=CB4856 RGLB=LB RGPL=illumina RGPU=PU RGSM=CB4856    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREAT

Then we mark duplicates

In [28]:
!java -Xmx{ramG}g -jar {softpath}/picard2.23.6/picard.jar MarkDuplicates I={SampleName}.RG.bam O={SampleName}.Dup.bam M={SampleName}.dedupMetrics REMOVE_DUPLICATES=true

INFO	2021-06-17 11:01:10	MarkDuplicates	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    MarkDuplicates -I CB4856.RG.bam -O CB4856.Dup.bam -M CB4856.dedupMetrics -REMOVE_DUPLICATES true
**********


11:01:11.116 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/Software/picard2.23.6/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Thu Jun 17 11:01:11 AST 2021] MarkDuplicates INPUT=[CB4856.RG.bam] OUTPUT=CB4856.Dup.bam METRICS_FILE=CB4856.dedupMetrics REMOVE_DUPLICATES=true    MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP=50000 MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=8000 SORTING_COLLECTION_SIZE_RATIO=0.25 TAG_DUPLICATE_SET_M

INFO	2021-06-17 11:03:49	MarkDuplicates	Read    23,000,000 records.  Elapsed time: 00:02:37s.  Time for last 1,000,000:    4s.  Last read position: V:5,682,199
INFO	2021-06-17 11:03:49	MarkDuplicates	Tracking 126877 as yet unmatched pairs. 26696 records in RAM.
INFO	2021-06-17 11:03:53	MarkDuplicates	Read    24,000,000 records.  Elapsed time: 00:02:41s.  Time for last 1,000,000:    3s.  Last read position: V:8,189,306
INFO	2021-06-17 11:03:53	MarkDuplicates	Tracking 131254 as yet unmatched pairs. 30493 records in RAM.
INFO	2021-06-17 11:03:57	MarkDuplicates	Read    25,000,000 records.  Elapsed time: 00:02:45s.  Time for last 1,000,000:    3s.  Last read position: V:10,651,728
INFO	2021-06-17 11:03:57	MarkDuplicates	Tracking 134890 as yet unmatched pairs. 33772 records in RAM.
INFO	2021-06-17 11:04:04	MarkDuplicates	Read    26,000,000 records.  Elapsed time: 00:02:52s.  Time for last 1,000,000:    7s.  Last read position: V:13,019,378
INFO	2021-06-17 11:04:04	MarkDuplicates	Tracking 135

Index bam

In [29]:
!samtools index {SampleName}.Dup.bam

Now we can realign

#### GATK realigments

GATK needs dictionary

In [43]:
!java -Xmx{ramG}g -jar {softpath}/picard2.23.6/picard.jar CreateSequenceDictionary R={ReferenceGenome} O={ReferenceGenome[:-3]}.dict

INFO	2021-06-17 11:30:00	CreateSequenceDictionary	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    CreateSequenceDictionary -R /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/data/Caenorhabditis_elegans.WBcel235.99.softmasked.fa -O /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/data/Caenorhabditis_elegans.WBcel235.99.softmasked.dict
**********


11:30:01.024 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/Software/picard2.23.6/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Thu Jun 17 11:30:01 AST 2021] CreateSequenceDictionary OUTPUT=/home/velazqam/Documents/Pr

In [48]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T RealignerTargetCreator -nt {Ncpu} -R {ReferenceGenome} -I {SampleName}.Dup.bam -o {SampleName}.Indel.intervals

INFO  13:18:28,146 HelpFormatter - ------------------------------------------------------------------------------------ 
INFO  13:18:28,149 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.8-1-0-gf15c1c3ef, Compiled 2018/02/19 05:43:50 
INFO  13:18:28,149 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  13:18:28,149 HelpFormatter - For support and documentation go to https://software.broadinstitute.org/gatk 
INFO  13:18:28,149 HelpFormatter - [Thu Jun 17 13:18:28 AST 2021] Executing on Linux 4.4.0-210-generic amd64 
INFO  13:18:28,149 HelpFormatter - OpenJDK 64-Bit Server VM 1.8.0_292-8u292-b10-0ubuntu1~16.04.1-b10 
INFO  13:18:28,153 HelpFormatter - Program Args: -T RealignerTargetCreator -nt 20 -R /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/data/Caenorhabditis_elegans.WBcel235.99.softmasked.fa -I CB4856.Dup.bam -o CB4856.Indel.intervals 
INFO  13:18:28,158 HelpFormatter - Executing as velazqam@kw60530 on Linux 4.4.0-210-generic amd64;

Realigment

In [49]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T IndelRealigner -R {ReferenceGenome} -I {SampleName}.Dup.bam -targetIntervals {SampleName}.Indel.intervals -o {SampleName}.realigned.bam

INFO  13:19:13,970 HelpFormatter - ------------------------------------------------------------------------------------ 
INFO  13:19:13,972 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.8-1-0-gf15c1c3ef, Compiled 2018/02/19 05:43:50 
INFO  13:19:13,973 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  13:19:13,973 HelpFormatter - For support and documentation go to https://software.broadinstitute.org/gatk 
INFO  13:19:13,973 HelpFormatter - [Thu Jun 17 13:19:13 AST 2021] Executing on Linux 4.4.0-210-generic amd64 
INFO  13:19:13,973 HelpFormatter - OpenJDK 64-Bit Server VM 1.8.0_292-8u292-b10-0ubuntu1~16.04.1-b10 
INFO  13:19:13,978 HelpFormatter - Program Args: -T IndelRealigner -R /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/data/Caenorhabditis_elegans.WBcel235.99.softmasked.fa -I CB4856.Dup.bam -targetIntervals CB4856.Indel.intervals -o CB4856.realigned.bam 
INFO  13:19:13,986 HelpFormatter - Executing as velazqam@kw60530 on Linux 4

Check files present and remove temporal files

In [53]:
!ls

CB4856.bam.bai	     CB4856.Indel.intervals	CB4856.realigned.bam
CB4856.dedupMetrics  CB4856.IndelIntervals.txt
CB4856.Dup.bam.bai   CB4856.realigned.bai


In [52]:
!rm {SampleName}.sam
!rm {SampleName}.bam
!rm {SampleName}.SF.bam
!rm {SampleName}.RG.bam
!rm {SampleName}.Dup.bam

Now is time for variant calling

#### Variant calling

In [3]:
##Move to analysis
os.chdir(path + '/analysis/')
##Create directory to place data
!mkdir -p VariantCalling
##Move to that directory
os.chdir(path + '/analysis/VariantCalling')

Use GATK unified genotyper thouhg most recent versions uses Haplotype callers (it should be better) due to actual base recalibration

In [11]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T UnifiedGenotyper -R {ReferenceGenome} -nt {Ncpu} -l INFO -glm BOTH -I {path}/analysis/Alignments/{SampleName}.realigned.bam -o {SampleName}.UG.vcf -mbq {minBaseQforVcf} -stand_call_conf {minVarCall}

INFO  17:02:12,156 HelpFormatter - ------------------------------------------------------------------------------------ 
INFO  17:02:12,158 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.8-1-0-gf15c1c3ef, Compiled 2018/02/19 05:43:50 
INFO  17:02:12,158 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  17:02:12,158 HelpFormatter - For support and documentation go to https://software.broadinstitute.org/gatk 
INFO  17:02:12,158 HelpFormatter - [Thu Jun 17 17:02:12 AST 2021] Executing on Linux 4.4.0-210-generic amd64 
INFO  17:02:12,158 HelpFormatter - OpenJDK 64-Bit Server VM 1.8.0_292-8u292-b10-0ubuntu1~16.04.1-b10 
INFO  17:02:12,161 HelpFormatter - Program Args: -T UnifiedGenotyper -R /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/data/Caenorhabditis_elegans.WBcel235.99.softmasked.fa -nt 20 -l INFO -glm BOTH -I /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/analysis/Alignments/CB4856.realigned.bam -o CB4856.UG.vcf

#### Now lets try with HaplotypeCaller

Emit confidence for vcf

In [16]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T HaplotypeCaller --emitRefConfidence GVCF -R {ReferenceGenome} -l INFO -I {path}/analysis/Alignments/{SampleName}.realigned.bam -o {SampleName}.g.vcf -mbq {minBaseQforVcf} -mmq {minMaqQforVcf} -stand_call_conf {minVarCall}

INFO  09:34:38,299 HelpFormatter - ------------------------------------------------------------------------------------ 
INFO  09:34:38,300 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.8-1-0-gf15c1c3ef, Compiled 2018/02/19 05:43:50 
INFO  09:34:38,301 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  09:34:38,301 HelpFormatter - For support and documentation go to https://software.broadinstitute.org/gatk 
INFO  09:34:38,301 HelpFormatter - [Sun Jun 20 09:34:38 AST 2021] Executing on Linux 4.4.0-210-generic amd64 
INFO  09:34:38,301 HelpFormatter - OpenJDK 64-Bit Server VM 1.8.0_292-8u292-b10-0ubuntu1~16.04.1-b10 
INFO  09:34:38,304 HelpFormatter - Program Args: -T HaplotypeCaller --emitRefConfidence GVCF -R /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/data/Caenorhabditis_elegans.WBcel235.99.softmasked.fa -l INFO -I /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/analysis/Alignments/CB4856.realigned.bam -o CB4856

INFO  09:49:40,658 ProgressMeter -      I:14979706              0.0    15.0 m         1488.2 w       14.9%   100.4 m      85.4 m 
INFO  09:50:10,660 ProgressMeter -      I:15065076              0.0    15.5 m         1537.8 w       15.0%   103.2 m      87.7 m 
INFO  09:50:40,662 ProgressMeter -      I:15066246              0.0    16.0 m         1587.4 w       15.0%   106.5 m      90.5 m 
INFO  09:51:10,664 ProgressMeter -      I:15066947              0.0    16.5 m         1637.0 w       15.0%   109.8 m      93.3 m 
INFO  09:51:40,667 ProgressMeter -       II:262002      1.5072434E7    17.0 m           67.0 s       15.3%   111.2 m      94.2 m 
INFO  09:52:10,669 ProgressMeter -       II:652570      1.5072434E7    17.5 m           69.0 s       15.7%   111.6 m      94.1 m 
INFO  09:52:40,671 ProgressMeter -       II:962259      1.5072434E7    18.0 m           71.0 s       16.0%   112.6 m      94.6 m 
INFO  09:53:10,673 ProgressMeter -      II:1059221      1.5072434E7    18.5 m           73

INFO  10:21:45,402 ProgressMeter -     III:7604431      3.0351855E7    47.0 m           92.0 s       37.8%     2.1 h      77.2 m 
INFO  10:22:15,404 ProgressMeter -     III:8129755      3.0351855E7    47.5 m           93.0 s       38.4%     2.1 h      76.3 m 
INFO  10:22:45,407 ProgressMeter -     III:8852946      3.0351855E7    48.0 m           94.0 s       39.1%     2.0 h      74.8 m 
INFO  10:23:15,409 ProgressMeter -     III:9605260      3.0351855E7    48.5 m           95.0 s       39.8%     2.0 h      73.2 m 
INFO  10:23:45,410 ProgressMeter -    III:10311882      3.0351855E7    49.0 m           96.0 s       40.5%     2.0 h      71.8 m 
INFO  10:24:15,412 ProgressMeter -    III:10923163      3.0351855E7    49.5 m           97.0 s       41.2%     2.0 h      70.8 m 
INFO  10:24:45,413 ProgressMeter -    III:11569274      3.0351855E7    50.0 m           98.0 s       41.8%   119.6 m      69.6 m 
INFO  10:25:15,415 ProgressMeter -    III:11962370      3.0351855E7    50.5 m           99

INFO  10:53:49,704 ProgressMeter -       V:5249995      6.1629485E7    79.0 m           76.0 s       66.7%   118.5 m      39.5 m 
INFO  10:54:19,706 ProgressMeter -       V:5839293      6.1629485E7    79.5 m           77.0 s       67.3%   118.2 m      38.7 m 
INFO  10:54:49,708 ProgressMeter -       V:6379258      6.1629485E7    80.0 m           77.0 s       67.8%   118.0 m      38.0 m 
INFO  10:55:19,711 ProgressMeter -       V:6788518      6.1629485E7    80.5 m           78.0 s       68.2%   118.0 m      37.5 m 
INFO  10:55:49,714 ProgressMeter -       V:7326831      6.1629485E7    81.0 m           78.0 s       68.8%   117.8 m      36.8 m 
INFO  10:56:19,716 ProgressMeter -       V:7382923      6.1629485E7    81.5 m           79.0 s       68.8%   118.4 m      36.9 m 
INFO  10:56:49,718 ProgressMeter -       V:7611090      6.1629485E7    82.0 m           79.0 s       69.0%   118.8 m      36.8 m 
INFO  10:57:19,721 ProgressMeter -       V:7643292      6.1629485E7    82.5 m           80

INFO  11:25:53,282 ProgressMeter -       X:7283496      8.2553665E7   111.0 m           80.0 s       89.6%     2.1 h      12.9 m 
INFO  11:26:23,284 ProgressMeter -       X:7921419      8.2553665E7   111.5 m           81.0 s       90.2%     2.1 h      12.1 m 
INFO  11:26:53,286 ProgressMeter -       X:8545386      8.2553665E7   112.0 m           81.0 s       90.8%     2.1 h      11.3 m 
INFO  11:27:23,288 ProgressMeter -       X:9175674      8.2553665E7   112.5 m           81.0 s       91.5%     2.0 h      10.5 m 
INFO  11:27:53,290 ProgressMeter -       X:9818274      8.2553665E7   113.0 m           82.0 s       92.1%     2.0 h       9.7 m 
INFO  11:28:23,292 ProgressMeter -      X:10487150      8.2553665E7   113.5 m           82.0 s       92.8%     2.0 h       8.8 m 
INFO  11:28:53,294 ProgressMeter -      X:11132064      8.2553665E7   114.0 m           82.0 s       93.4%     2.0 h       8.0 m 
INFO  11:29:23,297 ProgressMeter -      X:11773262      8.2553665E7   114.5 m           83

In [19]:
!java -Xmx{ramG}g -jar {softpath}/gatk-3.8.1.0/GenomeAnalysisTK.jar -T GenotypeGVCFs -R {ReferenceGenome} -nt {Ncpu} -l INFO -V {SampleName}.g.vcf -dt none -o {SampleName}.HC.vcf

INFO  12:03:19,468 HelpFormatter - ------------------------------------------------------------------------------------ 
INFO  12:03:19,470 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.8-1-0-gf15c1c3ef, Compiled 2018/02/19 05:43:50 
INFO  12:03:19,470 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  12:03:19,470 HelpFormatter - For support and documentation go to https://software.broadinstitute.org/gatk 
INFO  12:03:19,470 HelpFormatter - [Sun Jun 20 12:03:19 AST 2021] Executing on Linux 4.4.0-210-generic amd64 
INFO  12:03:19,470 HelpFormatter - OpenJDK 64-Bit Server VM 1.8.0_292-8u292-b10-0ubuntu1~16.04.1-b10 
INFO  12:03:19,473 HelpFormatter - Program Args: -T GenotypeGVCFs -R /home/velazqam/Documents/Projects/Jupyter/MappingBySequencing/Tutorial/data/Caenorhabditis_elegans.WBcel235.99.softmasked.fa -nt 20 -l INFO -V CB4856.g.vcf -dt none -o CB4856.HC.vcf 
INFO  12:03:19,478 HelpFormatter - Executing as velazqam@kw60530 on Linux 4.4.0-210-generic amd64; Op

#### Now SnpEff

In [20]:
##Move to analysis
os.chdir(path + '/analysis/')
##Create directory to place data
!mkdir -p VariantPrediction
##Move to that directory
os.chdir(path + '/analysis/VariantPrediction')

In [26]:
###Produce Annotations
!java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/snpEff.jar {SnpEffGen} -stats {SampleName}.HC.html {path}/analysis/VariantCalling/{SampleName}.HC.vcf > {SampleName}.HC.ann.vcf
!java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/snpEff.jar {SnpEffGen} -stats {SampleName}.UG.html {path}/analysis/VariantCalling/{SampleName}.UG.vcf > {SampleName}.UG.ann.vcf 

In [24]:
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - -s "," -e "." CHROM POS REF ALT QUAL GEN[{SampleName}].GT GEN[{SampleName}].DP GEN[*].AD "ANN[*].GENEID" "ANN[*].ALLELE" "ANN[*].EFFECT" "ANN[*].IMPACT" "ANN[*].GENE" "ANN[*].FEATURE" "ANN[*].FEATUREID" "ANN[*].BIOTYPE" "ANN[*].RANK" "ANN[*].HGVS_C" "ANN[*].HGVS_P" "ANN[*].CDNA_POS" "ANN[*].CDNA_LEN" "ANN[*].CDS_POS" "ANN[*].CDS_LEN" "ANN[*].AA_POS" "ANN[*].AA_LEN" "ANN[*].DISTANCE" "LOF[*].GENE" "LOF[*].GENEID" "NMD[*].GENE" "NMD[*].GENEID" > {SampleName}.HC.ann.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - -s "," -e "." CHROM POS REF ALT QUAL GEN[{SampleName}].GT GEN[{SampleName}].DP GEN[*].AD "ANN[*].GENEID" "ANN[*].ALLELE" "ANN[*].EFFECT" "ANN[*].IMPACT" "ANN[*].GENE" "ANN[*].FEATURE" "ANN[*].FEATUREID" "ANN[*].BIOTYPE" "ANN[*].RANK" "ANN[*].HGVS_C" "ANN[*].HGVS_P" "ANN[*].CDNA_POS" "ANN[*].CDNA_LEN" "ANN[*].CDS_POS" "ANN[*].CDS_LEN" "ANN[*].AA_POS" "ANN[*].AA_LEN" "ANN[*].DISTANCE" "LOF[*].GENE" "LOF[*].GENEID" "NMD[*].GENE" "NMD[*].GENEID" > {SampleName}.UG.ann.txt


In [30]:
## other approach
!echo "#High impact mutations" > {SampleName}.HC.ann.sort.txt
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'HIGH'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.HC.ann.sort.txt
!echo "#Moderate impact mutations" >> {SampleName}.HC.ann.sort.txt
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'MODERATE'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.HC.ann.sort.txt
!echo "#Low impact mutations" >> {SampleName}.HC.ann.sort.txt
!cat {SampleName}.HC.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'LOW'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.HC.ann.sort.txt

!echo "#High impact mutations" > {SampleName}.UG.ann.sort.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'HIGH'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.UG.ann.sort.txt
!echo "#Moderate impact mutations" >> {SampleName}.UG.ann.sort.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'MODERATE'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.UG.ann.sort.txt
!echo "#Low impact mutations" >> {SampleName}.UG.ann.sort.txt
!cat {SampleName}.UG.ann.vcf | perl {softpath}/SnpEff-5.0/snpEff/scripts/vcfEffOnePerLine.pl | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar filter "ANN[0].IMPACT has 'LOW'" | java -Xmx{ramG}g -jar {softpath}/SnpEff-5.0/snpEff/SnpSift.jar extractFields - CHROM POS REF ALT "ANN[*].GENE" "ANN[*].FEATUREID" "ANN[*].EFFECT" "LOF[*].GENE" >> {SampleName}.UG.ann.sort.txt

In [31]:
!echo "Unified Genotyper"
!head CB4856.UG.ann.sort.txt
!echo "Haplotype Caller"
!head CB4856.UG.ann.sort.txt

Unified Genotyper
#High impact mutations
CHROM	POS	REF	ALT	ANN[*].GENE	ANN[*].FEATUREID	ANN[*].EFFECT	LOF[*].GENE
I	6313	G	T	homt-1	Y74C9A.3.1	stop_gained	homt-1
I	59670	CA	C	Y48G1C.5	Y48G1C.5.1	frameshift_variant	Y48G1C.5
I	218097	A	AT	Y48G1BL.8	Y48G1BL.8.1	frameshift_variant	Y48G1BL.8
I	238696	A	AG	Y48G1BM.7	Y48G1BM.7.1	frameshift_variant	Y48G1BM.7
I	238954	A	T	Y48G1BM.7	Y48G1BM.7.1	stop_gained	Y48G1BM.7
I	642997	G	A	Y65B4A.2	Y65B4A.2c.1	splice_donor_variant&intron_variant	Y65B4A.2
I	664805	A	ACGACATC	Y65B4A.9	Y65B4A.9a.1	frameshift_variant	Y65B4A.9
I	688509	AG	A	ptp-5.3	Y18H1A.1.1	frameshift_variant	ptp-5.3
Haplotype Caller
#High impact mutations
CHROM	POS	REF	ALT	ANN[*].GENE	ANN[*].FEATUREID	ANN[*].EFFECT	LOF[*].GENE
I	6313	G	T	homt-1	Y74C9A.3.1	stop_gained	homt-1
I	59670	CA	C	Y48G1C.5	Y48G1C.5.1	frameshift_variant	Y48G1C.5
I	218097	A	AT	Y48G1BL.8	Y48G1BL.8.1	frameshift_variant	Y48G1BL.8
I	238696	A	AG	Y48G1BM.7	Y48G1BM.7.1	frameshift_variant	Y48G1BM.7
I	238954	A	T	Y48G1BM.7	Y48G1BM

Full report of SNPEff

<a href="analysis/VariantPrediction/CB4856.UG.html">Unified Genotyper results</a>

<a href="analysis/VariantPrediction/CB4856.HC.html">Haplotype Caller results</a>


In [34]:
#Analysis completed, going back to main directory
os.chdir(path)
##Create directory to place copy of results and check if files are there
!mkdir -p {path}/results
##Move to that directory
os.chdir( path + '/results' )

Make link to main results

In [36]:
!ln -s {path}/analysis/FastQC/*.zip .
!ln -s {path}/analysis/Alignments/*.realigned.bam .
!ln -s {path}/analysis/VariantCalling/*.vcf .
!ln -s {path}/analysis/VariantPrediction/*.txt .
!zip SnpEff.HC.zip *.HC.*.txt
!zip SnpEff.UG.zip *.UG.*.txt

ln: failed to create symbolic link './SRR2003569_1_fastqc.zip': File exists
ln: failed to create symbolic link './SRR2003569_2_fastqc.zip': File exists
ln: failed to create symbolic link './CB4856.realigned.bam': File exists
ln: failed to create symbolic link './CB4856.g.vcf': File exists
ln: failed to create symbolic link './CB4856.HC.vcf': File exists
ln: failed to create symbolic link './CB4856.UG.vcf': File exists
ln: failed to create symbolic link './CB4856.HC.ann.sort.txt': File exists
ln: failed to create symbolic link './CB4856.HC.ann.txt': File exists
ln: failed to create symbolic link './CB4856.HC.genes.txt': File exists
ln: failed to create symbolic link './CB4856.UG.ann.sort.txt': File exists
ln: failed to create symbolic link './CB4856.UG.ann.txt': File exists
ln: failed to create symbolic link './CB4856.UG.genes.txt': File exists
  adding: CB4856.HC.ann.sort.txt (deflated 88%)
  adding: CB4856.HC.ann.txt (deflated 92%)
  adding: CB4856.HC.genes.txt (deflated 86%)
  adding

#### Results

* <a href="results/SRR2003569_1_fastqc.zip">FastQC R1 report</a>
* <a href="results/SRR2003569_2_fastqc.zip">FastQC R2 report</a>
* <a href="results/CB4856.realigned.bam">Realigned bam</a>
* <a href="results/CB4856.HC.vcf">Haplotype Caller VCF</a>
* <a href="results/CB4856.UG.vcf">Unified Genotyper VCF</a>
* <a href="results/SnpEff.HC.zip">Haplotype Caller SnpEff annotations</a>
* <a href="results/SnpEff.UG.zip">Unified Genotyper SnpEff annotations</a>